In [1]:
import pandas as pd
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
train_data = pd.read_csv("/content/drive/MyDrive/dataset/word2vec/dataset/treino.csv")
train_data.sample(5)

,title,text,date,category,subcategory,link
42130,Crítica: Impossível passar indiferente por um ...,Não adianta procurar clemência nos olhos de Ma...,2015-01-30,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2015/01...
17303,"Em editoriais, imprensa estrangeira critica si...",O processo de impeachment que afastou Dilma Ro...,2016-05-13,mundo,NaN,http://www1.folha.uol.com.br/mundo/2016/05/177...
29201,Divulgação de e-mails revela a ironia e a falt...,Nos bastidores revelados por centenas de e-mai...,2015-02-07,mundo,NaN,http://www1.folha.uol.com.br/mundo/2015/07/165...
46617,Mostra de Sganzerla resgata 'Copacabana Mon Am...,"""O Terceiro Mundo vai explodir. Quem tiver sap...",2015-08-01,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2015/01...
59778,"Diferente de negros, violência policial contra...","Os policiais chegaram à casa em San Antonio, n...",2016-08-17,mundo,NaN,http://www1.folha.uol.com.br/mundo/2016/08/180...


In [ ]:
#execute just once
#!python -m spacy download pt_core_news_sm

In [4]:
nlp = spacy.load("pt_core_news_sm")

In [6]:
text = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(text)
doc

Rio de Janeiro é uma cidade maravilhosa

In [7]:
type(doc[2])

spacy.tokens.token.Token

In [8]:
texts_to_treatment = (titles.lower() for titles in train_data["title"])

In [9]:
def treat_texts(doc):
    valid_tokens = []
    for token in doc:
        is_valid = not token.is_stop and token.is_alpha
        if is_valid:
            valid_tokens.append(token.text)

    if len(valid_tokens) > 2:
        return  " ".join(valid_tokens)

text = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(text)
treat_texts(doc)

'Rio Janeiro cidade maravilhosa'

In [10]:
text = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(text)
treat_texts(doc)

'Rio Janeiro cidade maravilhosa'

In [12]:
from time import time

t0 = time()
treated_texts = [treat_texts(doc) for doc in nlp.pipe(texts_to_treatment,
                                                        batch_size = 1000,
                                                        n_process = -1)]
tf = time() - t0
print(tf/60)

1.5841255068778992


In [17]:
treated_titles = pd.DataFrame({"titulo": treated_texts})
treated_titles.head()

,titulo
0,None
1,saberemos verdade diretor série robert durst
2,clubes pediram saída gallo coordenação cbf sub...
3,limitar gasto público exigirá medidas compleme...
4,antonio carlos trommer resende rádio literatur...


In [14]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [20]:
print(len(treated_titles))
treated_titles = treated_titles.dropna().drop_duplicates()
print(len(treated_titles))

64000
60086


In [21]:
list_tokens_list = [title.split(" ") for title in treated_titles.titulo]

In [24]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(list_tokens_list, progress_per=5000)